In [1]:
import os
import sys
current_directory = os.getcwd()
if not os.path.exists(current_directory + '\\experimental_data'):
    os.makedirs(current_directory + '\\experimental_data')
    os.makedirs(current_directory + '\\experimental_data\\plots')
    os.makedirs(current_directory + '\\experimental_data\\results')
sys.path.append(current_directory.replace('experiments', ''))
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.set_loglevel('WARNING')
from run import execute_simulation, create_experiment_snapshots, execute_ai_model, single_mtd_simulation, mtd_ai_simulation, multiple_mtd_simulation, specific_multiple_mtd_simulation
from mtdnetwork.mtd.completetopologyshuffle import CompleteTopologyShuffle
from mtdnetwork.mtd.ipshuffle import IPShuffle
from mtdnetwork.mtd.hosttopologyshuffle import HostTopologyShuffle
from mtdnetwork.mtd.portshuffle import PortShuffle
from mtdnetwork.mtd.osdiversity import OSDiversity
from mtdnetwork.mtd.servicediversity import ServiceDiversity
from mtdnetwork.mtd.usershuffle import UserShuffle
from mtdnetwork.mtd.osdiversityassignment import OSDiversityAssignment
import logging
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt
from run_experiment import Experiment

logging.basicConfig(format='%(message)s', level=logging.INFO)

In [2]:
# Learning Parameters
epsilon = 1.0  # exploration rate

# Simulator Settings
start_time = 0
finish_time = 3000
mtd_interval = [200]
network_size = [75]
total_nodes = 300
new_network = True
model = 'all_features'

trial = 3000

mtd_strategies = [
    CompleteTopologyShuffle,
    # HostTopologyShuffle,
    IPShuffle,
    OSDiversity,
    # PortShuffle,
    # OSDiversityAssignment,
    ServiceDiversity,
    # UserShuffle
]

result_head_path = '/Users/williamho/Documents/GitHub/MTDSim'



In [3]:
# result = Experiment(epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network, model, trial)
# result.raw_result_stats_pipeline('nomtd',run_trial = True, stats_type = 'median')
# for scheme in result.schemes:
#     result.raw_result_stats_pipeline(scheme,run_trial = True, stats_type = 'median')

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

class RadarPlot(Experiment):
    def __init__(self, epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network,  model, trial, result_head_path):
        super().__init__(epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network,  model, trial, result_head_path)

    def plot_n_schemes(self, schemes_data):
        """
        Plots multiple schemes on a radar chart.
        
        :param schemes_data: A dictionary where keys are scheme names and values are dictionaries of metrics.
        """
        scheme_names = list(schemes_data.keys())
        first_scheme = schemes_data[scheme_names[0]]
        labels = list(first_scheme.keys())
        num_vars = len(labels)

        # Compute angle for each axis
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle

        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

        # Use colormap for colors
        colors = plt.cm.tab20.colors
        legend_patches = []

        for i, scheme_name in enumerate(scheme_names):
            metrics_values = schemes_data[scheme_name]
            values = list(metrics_values.values())
            values += values[:1]  # Close the circle

            # Plot data
            ax.fill(angles, values, color=colors[i % len(colors)], alpha=0.25)
            ax.plot(angles, values, color=colors[i % len(colors)], linewidth=2, label=scheme_name)

            # Add score labels
            for j in range(num_vars):
                angle = angles[j]
                value = values[j]
                ax.text(angle, value + 0.05, f'{value:.2f}', horizontalalignment='center', size=10, color='black')

        # Add labels
        ax.set_yticklabels([])
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, rotation=45, ha='right')

        # Create legend
        legend_patches = [Patch(color=colors[i % len(colors)], label=scheme_name) for i, scheme_name in enumerate(scheme_names)]
        plt.legend(handles=legend_patches, loc='upper right', bbox_to_anchor=(1.1, 1.1))

        plt.show()



In [5]:

# radar = RadarPlot(epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network,  model, trial, result_head_path)
# schemes_data = radar.multiple_scaled_pipeline(['mtd_ai', 'random'],run_trial = False)
# radar.plot_n_schemes(schemes_data)


In [6]:
static_features = ["host_compromise_ratio", "exposed_endpoints", "attack_path_exposure",  "overall_asr_avg", "roa", "shortest_path_variability", "risk"]
time_features = ["mtd_freq", "overall_mttc_avg", "time_since_last_mtd"]
features =  static_features + time_features

for model in features:
    print(model)
    result = Experiment(epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network, model, trial, result_head_path)
    result.run_trials(model)
    


host_compromise_ratio
Trial_ 0
host_compromise_ratio


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


On Mac


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 1
host_compromise_ratio
On Mac


...

mtd_ai
Trial_ 236
host_compromise_ratio
On Mac


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 237
host_compromise_ratio
On Mac


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 238
host_compromise_ratio
On Mac


In [ ]:
model = "all_features"
result = Experiment(epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network, model, trial, result_head_path)
result.run_trials(model)

Trial_ 0
all_features


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


On Mac


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 1
all_features
On Mac


...

mtd_ai
Trial_ 281
all_features
On Mac


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 282
all_features
On Mac


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 283
all_features
On Mac
